<a href="https://colab.research.google.com/github/Aancha2000l/Sentiment_Analysis/blob/main/ML_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install contractions
!pip install vaderSentiment
!pip install gitpython

In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [35]:
#web scarpping excracted the html file
 
url ='https://inshorts.com/en/read/sports'
news_data = []
news_category =url.split("/")[-1]
data=requests.get(url)
soup = BeautifulSoup(data.content)
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<style>
    /* The Modal (background) */
    .modal_contact {
        display: none; /* Hidden by default */
        position: fixed; /* Stay in place */
        z-index: 8; /* Sit on top */
        left: 0;
        top: 0;
        width: 100%; /* Full width */
        height: 100%;
        overflow: auto; /* Enable scroll if needed */
        background-color: rgb(0,0,0); /* Fallback color */
        background-color: rgba(0,0,0,0.4); /* Black w/ opacity */
    }

    /* Modal Content/Box */
    .modal-content {
        background-color: #fefefe;
        margin: 15% auto;
        padding: 20px !important;
        padding-top: 0 !important;
        /* border: 1px solid #888; */
        text-align: center;
        position: relative;
        border-radius: 6px;
    }

    /* The Close Button */
    .close {
      left: 90%;
      color: #aaa;
      float: right;
      font-size: 28px;
      font-weight: bold;
    /* positio

In [36]:
urls = ['https://inshorts.com/en/read/sports','https://inshorts.com/en/read/world','https://inshorts.com/en/read/politics']
def build_dataset(urls):
  news_data = []
  for url in urls:
    news_category =url.split("/")[-1]
    data = requests.get(url)
    soup = BeautifulSoup(data.content)
  
    news_articles = [{'news_headline': headline.find('span', attrs={"itemprop":"headline"}).string,
                      'news_article': article.find('div', attrs={"itemprop":"articleBody"}).string,
                      'news_category': news_category}
                     
                      for headline,article in zip(soup.find_all('div',class_=["news-card-title news-right-box"]),
                                                  soup.find_all('div',class_=["news-card-content news-right-box"]))]
    news_articles = news_articles[0:20]
    news_data.extend(news_articles)

  df = pd.DataFrame(news_data)
  df = df[['news_headline','news_article','news_category']]  
  return df                

In [37]:
df = build_dataset(urls)
df.tail()

,news_headline,news_article,news_category
55,EC defers bypolls in 3 Parliamentary and 8 Ass...,In view of the surging COVID-19 cases in the c...,politics
56,"After Amphan, WB facing Mamata-phan: JP Nadda ...",BJP National President JP Nadda said that last...,politics
57,Priority is to end senseless violence: WB Guv ...,After Mamata Banerjee took oath as the CM of W...,politics
58,"Elections over, looting resumes: Rahul Gandhi ...",Congress leader Rahul Gandhi said that looting...,politics
59,Nationwide lockdown only option left to check ...,Rajasthan Chief Minister Ashok Gehlot on Thurs...,politics


In [38]:
#PREPROCESSING

import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
len(stopword_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


177

In [39]:
# removing html tags
def html_tag(text):
  soup =BeautifulSoup(text,"html.parser")
  new_text = soup.get_text()
  return new_text

html_tag('<html><h2> Some important info  </h2><html>')  
' Some important info '

' Some important info '

In [40]:
#Expand Contractions
!pip install contractions
import contractions
def con(text):
  expand = contractions.fix(text)
  return expand

con("Y'all can't expand I'd think")

'you all can not expand I would think'

In [41]:
# removed special character

import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]'
  text = re.sub(pattern,'',text)
  return text


In [42]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer =ToktokTokenizer()

In [43]:
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

In [44]:
#applying to actual database

df.news_headline = df.news_headline.apply(lambda x:x.lower())
df.news_article = df.news_article.apply(lambda x:x.lower())

df.news_headline = df.news_headline.apply(html_tag)
df.news_article = df.news_article.apply(html_tag)
df.news_headline = df.news_headline.apply(con)
df.news_article = df.news_article.apply(con)
df.news_headline = df.news_headline.apply(remove_sp)
df.news_article = df.news_article.apply(remove_sp)
df.news_headline = df.news_headline.apply(remove_stopwords)
df.news_article = df.news_article.apply(remove_stopwords)

df.head()

,news_headline,news_article,news_category
0,cricketer vedas sister passes away due covid19...,india women cricketer veda krishnamurthys sist...,sports
1,prakash padukone hospitalised testing positive...,actress deepika padukones father former badmin...,sports
2,I happy call ravindra jadeja jadeja bhogle cal...,reacting video wherein commentator harsha bhog...,sports
3,police launch raids locate sushil questioning ...,delhi police launched raids locate twotime oly...,sports
4,bangladesh cricket board share insensitive pos...,bangladesh cricket board shared post twitter 3...,sports


In [45]:
# dataset labeling and processing
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs =  SentimentIntensityAnalyzer()
df['compound'] = df['news_headline'].apply(lambda x: vs.polarity_scores(x)['compound'])
df.head()

,news_headline,news_article,news_category,compound
0,cricketer vedas sister passes away due covid19...,india women cricketer veda krishnamurthys sist...,sports,-0.5994
1,prakash padukone hospitalised testing positive...,actress deepika padukones father former badmin...,sports,0.5574
2,I happy call ravindra jadeja jadeja bhogle cal...,reacting video wherein commentator harsha bhog...,sports,0.5719
3,police launch raids locate sushil questioning ...,delhi police launched raids locate twotime oly...,sports,-0.7269
4,bangladesh cricket board share insensitive pos...,bangladesh cricket board shared post twitter 3...,sports,0.0772


In [46]:
# data finalization 
def predict(comp):
  comp=float(comp)
  if (comp>0):
    return 'positive'
  else:
    return 'negative'
df['type_pred'] = df['compound'].apply(predict)
df.head()

,news_headline,news_article,news_category,compound,type_pred
0,cricketer vedas sister passes away due covid19...,india women cricketer veda krishnamurthys sist...,sports,-0.5994,negative
1,prakash padukone hospitalised testing positive...,actress deepika padukones father former badmin...,sports,0.5574,positive
2,I happy call ravindra jadeja jadeja bhogle cal...,reacting video wherein commentator harsha bhog...,sports,0.5719,positive
3,police launch raids locate sushil questioning ...,delhi police launched raids locate twotime oly...,sports,-0.7269,negative
4,bangladesh cricket board share insensitive pos...,bangladesh cricket board shared post twitter 3...,sports,0.0772,positive


In [47]:
df.to_csv('news.csv',index=False)

In [48]:
df = pd.read_csv('/content/news.csv')
df


,news_headline,news_article,news_category,compound,type_pred
0,cricketer vedas sister passes away due covid19...,india women cricketer veda krishnamurthys sist...,sports,-0.5994,negative
1,prakash padukone hospitalised testing positive...,actress deepika padukones father former badmin...,sports,0.5574,positive
2,I happy call ravindra jadeja jadeja bhogle cal...,reacting video wherein commentator harsha bhog...,sports,0.5719,positive
3,police launch raids locate sushil questioning ...,delhi police launched raids locate twotime oly...,sports,-0.7269,negative
4,bangladesh cricket board share insensitive pos...,bangladesh cricket board shared post twitter 3...,sports,0.0772,positive
5,take jet witness bodies streets india slater a...,australian cricketerturnedcommentator michael ...,sports,0.0000,negative
6,csk member reveals dhoni delayed return says l...,csk captain ms dhoni informed teammates last p...,sports,-0.2732,negative
7,fail complete ipl 2021 loss close 2500 crore s...,bcci president sourav ganguly said ipl 2021 no...,sports,-0.7003,negative
8,24yrold wrestler beaten death delhi sushil den...,24yearold wrestler allegedly beaten death two ...,sports,-0.8591,negative
9,fan spots exfootballers daughters face clouds ...,daughters death former uk footballer ashley ca...,sports,-0.1027,negative


In [49]:
# dropping not required data and re-indexing the remaining data
df = df.drop(columns=['news_category'], axis=1)
df = df.drop(columns=['news_article'], axis=1)
df = df.set_index(np.arange(len(df)))
df

,news_headline,compound,type_pred
0,cricketer vedas sister passes away due covid19...,-0.5994,negative
1,prakash padukone hospitalised testing positive...,0.5574,positive
2,I happy call ravindra jadeja jadeja bhogle cal...,0.5719,positive
3,police launch raids locate sushil questioning ...,-0.7269,negative
4,bangladesh cricket board share insensitive pos...,0.0772,positive
5,take jet witness bodies streets india slater a...,0.0000,negative
6,csk member reveals dhoni delayed return says l...,-0.2732,negative
7,fail complete ipl 2021 loss close 2500 crore s...,-0.7003,negative
8,24yrold wrestler beaten death delhi sushil den...,-0.8591,negative
9,fan spots exfootballers daughters face clouds ...,-0.1027,negative


In [50]:
#Splitting the data into train and test data

from sklearn.model_selection import train_test_split
x = df.iloc[:,0].values
y = df.iloc[:,2].values
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=100)

In [51]:
# TFIDF Vectoriztion
# Creating the model using Multinomial Naive Bayes Agorithm
# Model is created using Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model=Pipeline([('TFIDF',TfidfVectorizer()),('model',MultinomialNB())])
model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('TFIDF',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [52]:
y_pred=model.predict(x_test)
y_pred

array(['negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative'],
      dtype='<U8')

In [53]:
y_test

array(['positive', 'negative', 'negative', 'positive', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'positive', 'negative', 'negative'],
      dtype=object)

In [54]:
#evaluation

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
accuracy_score(y_pred,y_test)

0.8

In [55]:
confusion_matrix(y_pred,y_test)

array([[12,  3],
       [ 0,  0]])

In [56]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    negative       1.00      0.80      0.89        15
    positive       0.00      0.00      0.00         0

    accuracy                           0.80        15
   macro avg       0.50      0.40      0.44        15
weighted avg       1.00      0.80      0.89        15



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
#predicting for new value

text=['pic csks spreadout field defending 2 vs pollard last ball goes viral']
model.predict(text)

array(['negative'], dtype='<U8')

In [58]:
# storing the model using joblib
 
import joblib
joblib.dump(model,'sentiment analysis')

['sentiment analysis']

In [59]:
# Using Streamit
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

In [60]:
%%writefile app.py
import streamlit as st
import sklearn
import joblib
model=joblib.load('sentiment analysis')
st.title('Sentiment Analysis ')
ip=st.text_input('Enter the text:')
op=model.predict([ip])
if st.button('Predict'):
  st.title(op[0])

Overwriting app.py


In [61]:
#for getting url
!nohup streamlit run app.py &
url=ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://022979016f65.ngrok.io'

In [62]:
!pip install pipreqs